In [2]:
import json
f = open('./output.json')
data = json.load(f)

In [3]:
def fontsize(flag):
  if flag == 'large': return 50
  elif flag == 'heading': return 40
  elif flag == 'small': return 30
  elif flag == 'text': return 20
  elif flag == 'span': return 10
  else: return 5

def tag(flag):
  # return 'p'
  if flag == 'large': return 'h1'
  elif flag == 'heading': return 'h2'
  elif flag == 'small': return 'h3'
  elif flag == 'text': return 'h4'
  elif flag == 'span': return 'p'
  else: return 'p'

```TODO:``` A following marker word-by-word when speaking

In [13]:
# Open the HTML file in write mode
with open('interface.html', 'w') as html_file:
    # Write the HTML file header
    html_file.write("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Output HTML</title>
        <style>
                    span {font-family: sans-serif}
                    .spanhoverable, .divhoverable, .divhoverable_sub  {
                        position: relative;
                    }
                    .spanhoverable:hover {background-color: green}
                    .divhoverable_sub:hover .ptoolbar, .divhoverable_sub .ptoolbar:hover, .spanhoverable:hover .toolbar, .spanhoverable .toolbar:hover {display: flex}
                    .toolbar {
                        display: none;
                        position: absolute;
                        margin-top: -20px;
                        left: 0;
                        top: 0;
                        z-index: 9;
                    }
                    .ptoolbar {
                        display: none;
                        position: absolute;
                        margin-top: -20px;
                        right: 0;
                        bottom: -20px;
                        z-index: 9;
                    }
                    .hidden {
                        display: none
                    }

        </style>
        <script>
                    function playaudio(address) {
                        var audio = new Audio(address);
                        audio.play();
                    }

                    function show_summary(id) {
                        document.querySelector(`#original_${id}`).classList.add('hidden')
                        document.querySelector(`#summary_${id}`).classList.remove('hidden')
                    }
                    function show_original(id) {
                        document.querySelector(`#summary_${id}`).classList.add('hidden')
                        document.querySelector(`#original_${id}`).classList.remove('hidden')
                    }

                    function separate(id) {
                        document.querySelectorAll(`#original_${id} .bt_to_separate`).forEach(br => br.classList.toggle('hidden'))
                        document.querySelectorAll(`#summary_${id} .bt_to_separate`).forEach(br => br.classList.toggle('hidden'))
                    }

        </script>
    </head>
    <body>
    """)

    for segment in data:
        if segment["flag"] == "image":
            html_file.write(f"""
                <img
                    src="{segment['image']['image']}"
                    width="{segment['image']['bbox'][2]-segment['image']['bbox'][0]}"
                    height="{segment['image']['bbox'][3]-segment['image']['bbox'][1]}"
                >
            """)
        elif segment["flag"] != "span":
            html_file.write(f"""<{tag(segment["flag"])} class="divhoverable">""")
            # write the original
            index = -1
            html_file.write(f"""<div id="original_{segment['id']}" class="divhoverable_sub original">""")
            if 'summery' in segment and len(segment["summery_parts"])>0: html_file.write(f"""<pre style="display: contents;">original</pre>""")
            for part in segment["parts"]:
                index+=1

                sub_voice_address = f"./voices/{segment['id']}.part.{index}.wav"
                if len(segment["parts"]) == 1 : sub_voice_address = f"./voices/{segment['id']}.original.wav"
                html_file.write(f"""
                    <span
                    class="spanhoverable"
                    style="
                        font-weight: 100;
                        color: rgb({(segment["style"]["color"] >> 16) & 255}, {(segment["style"]["color"] >> 8) & 255}, {segment["style"]["color"] & 255});
                    ">
                        <span class="original_text_part">{part}</span>
                        <br class="bt_to_separate hidden"/>
                        <br class="bt_to_separate hidden"/>
                        <div class="toolbar">
                            <button onclick="playaudio('{sub_voice_address}')">speak</button>
                        </div>
                    </span>
                """)
            html_file.write(f"""<div class="ptoolbar">""")
            html_file.write(f"""<button onclick="playaudio('./voices/{segment['id']}.original.wav')">speak</button>""")
            if 'summery_parts' in segment and len(segment["summery_parts"])>0: html_file.write(f"""<button onclick="show_summary({segment['id']})">summarise</button>""")
            if 'parts' in segment and len(segment["parts"])>1: html_file.write(f"""<button onclick="separate({segment['id']})">separate</button>""")
            html_file.write(f"""</div>""")
            
            html_file.write(f"""</div>""")

            # summary
            if 'summery_parts' in segment:
                index = -1
                html_file.write(f"""</br>""")
                html_file.write(f"""<div id="summary_{segment['id']}" class="divhoverable_sub summary hidden">""")
                html_file.write(f"""<pre style="display: contents;">summary</pre>""")
                for part in segment["summery_parts"]:
                    index+=1
                    sub_summary_voice_address = f"./voices/{segment['id']}.summary_part.{index}.wav"
                    if len(segment["summery_parts"]) == 1 : sub_summary_voice_address = f"./voices/{segment['id']}.summary.wav"
                    html_file.write(f"""
                        <span
                        class="spanhoverable"
                        style="
                            font-weight: 100;
                            color: rgb({(segment["style"]["color"] >> 16) & 255}, {(segment["style"]["color"] >> 8) & 255}, {segment["style"]["color"] & 255});
                        ">
                            <span class="original_text_part">{part}</span>
                            <br class="bt_to_separate hidden"/>
                            <br class="bt_to_separate hidden"/>
                            <div class="toolbar">
                                <button onclick="playaudio('{sub_summary_voice_address}')">speak</button>
                            </div>
                        </span>
                    """)
                html_file.write(f"""<div class="ptoolbar">""")
                html_file.write(f"""<button onclick="playaudio('./voices/{segment['id']}.summary.wav')">speak</button>""")
                if 'summery_parts' in segment and len(segment["summery_parts"])>0: html_file.write(f"""<button onclick="show_original({segment['id']})">original</button>""")
                if 'summery_parts' in segment and len(segment["summery_parts"])>1: html_file.write(f"""<button onclick="separate({segment['id']})">separate</button>""")
                html_file.write(f"""</div>""")

                html_file.write(f"""</div>""")
                
                html_file.write(f"""</{tag(segment["flag"])}>""")


    # Write the HTML file footer
    html_file.write("""
    </body>
    </html>
    """)

print("HTML file has been generated successfully.")

HTML file has been generated successfully.
